In [5]:
import numpy as np
import tensorflow as tf
import pandas as pd
from os import listdir
from os.path import isfile, join

In [19]:
trainFileTest = pd.read_csv("data/train/output_23.csv")
oil = pd.read_csv("data/oil.csv")
stores = pd.read_csv("data/stores.csv")
holidays_events = pd.read_csv("data/holidays_events.csv")
transactions = pd.read_csv("data/transactions.csv")
mypath = "data/train/"
trainFileList = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [7]:
print(oil.isnull().sum())
oil = oil.fillna(method="pad")
oil = oil.fillna(value=93.14, limit=1)
oil.head()

date           0
dcoilwtico    43
dtype: int64


,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [8]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [20]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [ ]:
print(transactions["indexColumn"][0])
trainFileTest.head(
)

In [12]:
mainTrain = pd.merge(left=trainFileTest, right=stores, how="left", left_on="store_nbr", right_on="store_nbr")
mainTrain.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster
0,22000000,2014-04-07,39,215352,38.0,False,Cuenca,Azuay,B,6
1,22000001,2014-04-07,39,215356,1.0,False,Cuenca,Azuay,B,6
2,22000002,2014-04-07,39,215370,4.0,False,Cuenca,Azuay,B,6
3,22000003,2014-04-07,39,215896,11.0,False,Cuenca,Azuay,B,6
4,22000004,2014-04-07,39,218728,4.0,False,Cuenca,Azuay,B,6
